In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import xgboost as xg
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as MSE
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
import os
import PIL.Image
import cv2
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [53]:
parent_folder = './'

In [54]:
labels_df = pd.read_csv(parent_folder+'labels_df.csv')
sort_idx = np.argsort( labels_df.slide_id.values )
labels_df = labels_df.loc[sort_idx]
labels_df.reset_index(inplace=True, drop=True)
gby_temp = list(labels_df.groupby('biopsy_id',sort=False))
biopsy_idx_to_slide_idx = np.array( [ l[1].index.values for l in gby_temp ], dtype=object )

In [55]:
labels_df.head()

,slide_id,biopsy_id,stage
0,0000459a-838d-4865-8bbf-ea66f2e5ee4d,acf982cf-42f1-4837-b47e-a98541cb65fa,3
1,0001669a-9515-42f2-926b-48f153fd9643,2025b0a1-d639-46a7-a2b6-58ecc32f054b,0
2,00018c3b-e8c7-49db-9831-7802bde48706,3e8bd0fd-e179-4aeb-b714-91a9453831fa,1
3,00027644-c853-4df4-abda-c13619f1e688,396ed629-7931-47bb-b4d5-57600ff46108,1
4,00046960-74ec-4750-b972-c4bf78eb40d6,45ceba23-f582-4c4a-bcb7-1716401afb68,1


In [56]:
len(labels_df["slide_id"].value_counts())

45701

In [57]:
len(labels_df["biopsy_id"].value_counts())

2721

In [58]:
# for each biopsy make a list of the slides id's by re-shaping the df. 
list_df = labels_df.groupby(["biopsy_id"]).agg({'slide_id': lambda x: x.tolist()}).reset_index()
list_df.head()

,biopsy_id,slide_id
0,00047e6d-cf9e-41f8-8901-eb9b0fe155a6,"[028ad74f-ebd7-45e5-92da-b0f2ded35fdf, 0d67764..."
1,0028943f-8565-4ea7-89df-0a53f37db9ca,"[0012e24b-3dd3-4226-a12e-aeeae8d18df4, 10117ed..."
2,0034f45e-6adb-49da-afbc-1cb89c60f1bc,"[06820b59-9f49-4ac2-8c42-2fa4990dcc5b, 3343d1f..."
3,0034fac2-6ef7-4337-a5bc-05065d5d4d68,"[875fc427-e1e3-4f1b-a80b-0c5db2ff4fd4, 91ebcd3..."
4,00437712-9cc8-40b4-a277-b2203ce41878,"[01ee5a84-21e5-4151-9fd8-31b145e2e9ce, 03222bf..."


In [9]:
def re_pack(x):
    biopsy_bag = np.zeros((1,2048))
    for slide in x["slide_id"]:
        try:
            slide_bag = np.load("../save_resnet_embeddings_level4_bags_bracs/resnet50_bracs_"+slide+".npy")
            biopsy_bag = np.vstack((biopsy_bag,slide_bag)).astype(np.float16)
            # append to biopsy bag.
        except:
            pass
    # save biopsy bag with biopsy id as the name.
    biopsy_bag = biopsy_bag[1:,:]
    np.save("../save_resnet_embeddings_level4_biopsy_bags_bracs_float16/"+x["biopsy_id"]+".npy",biopsy_bag)
     
# lambda function for entire set of all bags.
list_df.apply(lambda x: re_pack(x) ,axis=1) # uncomment top do the packing!


0      None
1      None
2      None
3      None
4      None
       ... 
881    None
882    None
883    None
884    None
885    None
Length: 886, dtype: object

#### run only once 